In [5]:
# !pip install dico/fr_core_news_md-3.7.0-py3-none-any.whl
# !pip install pandas spacy matplotlib
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install regex

import pandas as pd
import matplotlib.pyplot as plt
import re
import spacy
import os
from urllib.parse import urlparse
from fuzzywuzzy import fuzz
import platform

# Charger le modèle spaCy
nlp = spacy.load("fr_core_news_md")


list_mois = [
    "Mai", 
    "Juin", 
    "Juillet",
    "Aout"
    ]

for mois in list_mois:

    if platform.system() == 'Windows':
        folder = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Scrapping\\Datasets\\{mois}'
    else:
        folder = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Scrapping/Datasets/{mois}'


    # Fonction pour obtenir tous les fichiers CSV dans un dossier et ses sous-dossiers
    def get_all_csv_files(folder):
        csv_files = []
        for root, _, files in os.walk(folder):
            for file in files:
                if file.endswith('.csv'):
                    csv_files.append(os.path.join(root, file))
        print(f"Les traitements sont en cours pour le mois de {mois}, cela pourrait prendre quelques minutes...")
        return csv_files

    file_paths = get_all_csv_files(folder)

    # Charger et fusionner les fichiers
    df_list = []
    for file in file_paths:
        try:
            # Vérifier si le fichier est vide avant de lire
            if os.path.getsize(file) > 0:
                df = pd.read_csv(file)
                if not df.empty:
                    df_list.append(df)
                else:
                    print(f"Le fichier {file} est vide et a été ignoré.")
            else:
                print(f"Le fichier {file} est vide et a été ignoré.")
        except pd.errors.EmptyDataError:
            print(f"Le fichier {file} est mal formaté et a été ignoré.")

    # Fusionner les DataFrames non vides
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
        print(f"Restants : {len(df)}")
    else:
        print("Aucun fichier valide n'a été trouvé.")


    print("2.2.	Convertir toutes les données en minuscules pour faciliter le traitement des données textuelles")

    # Convertir toutes les valeurs en minuscules pour les colonnes de type str, sauf pour les colonnes spécifiées
    columns_to_exclude = ['superficie', 'nb_pieces', 'nb_salle_de_bain', 'scraping_date', 'link']
    df = df.apply(lambda col: col.str.lower() if col.name not in columns_to_exclude and col.dtype == 'object' else col)

    # garder les lignes dont les descriptions contiennent plus de 30 mots
    seuil_nombre_tokens = 15
    df = df[df['description'].str.split().str.len() >= seuil_nombre_tokens]

    print(f'Restants : {len(df)}')


    print("2.3. Création des variables « site » et « date »")

    # Fonction pour extraire le nom de domaine à partir de l'URL
    def extract_site(link):
        try:
            return urlparse(link).netloc
        except:
            return None

    # Ajouter la colonne 'site' en extrayant le nom de domaine de la colonne 'link'
    if 'link' in df.columns:
        df['site'] = df['link'].apply(extract_site)
        

    if 'scraping_date' in df.columns:
        df = df.drop(columns=['scraping_date'])
    else:
        print("La colonne 'scraping_date' n'existe pas dans le DataFrame.")



    print("2.4.	Suppression des lignes non pertinentes pour l’étude (vente, entrepôt, terrains, etc.)")

    # Supprimer les lignes où 'price' contient des indications de loyer journalier
    daily_and_no_location = r'\b(jour|vente|vendre|nuit|entrepôt|hôtel|entrepot|hotel|terrain|triplex|résidence|residence|residentiel|résidentiel|golf|ambassde|zone4|zone 4|haut stanting|luxueux|luxueuse|luxueuses|résidences|residences|residentiels|résidentiels)\b'

    # Créer une nouvelle colonne temporaire qui concatène 'description' et 'title'
    df['concat_description_title'] = df['description'].fillna('') + ' ' + df['title'].fillna('') + ' ' + df['price'].fillna('') + ' ' + df['localisation'].fillna('')

    # Filtrer les lignes qui ne contiennent pas les mots-clés
    df = df[~df['concat_description_title'].str.contains(daily_and_no_location, case=False, na=False)]

    # Supprimer la colonne temporaire après le filtrage
    df = df.drop(columns=['concat_description_title'])

    # Supprimer les lignes où title est compose de plusieurs logements
    multiple_properties_pattern = r'\b(et|appartements|studios|maisons|villas|bureaux|penthouses)\b'

    # Filtrer les lignes qui ne contiennent pas les mots-clés
    df = df[~df['title'].str.contains(multiple_properties_pattern, case=False, na=False)]


    # Afficher le nombre de lignes restantes après le filtrage
    print(f'Restants : {len(df)}')

    print("2.5.	Numérisation des variables quantitatives (nombre de pièces, nombre de salle de bain, loyer_mensuel et superficie en m2)")

    # Fonction pour nettoyer et convertir les valeurs de 'price' en format numérique
    def clean_price(price):
        # Extraire les chiffres de la chaîne de caractères
        price = re.sub(r'[^\d]', '', str(price))
        # Convertir en entier
        return float(price) if price.isdigit() else None
    # Appliquer la fonction de nettoyage sur la colonne 'price', et traduction du nom de la variable
    if 'price' in df.columns:
        df['loyer_mensuel'] = df['price'].apply(clean_price)
        df.drop(columns=['price'], inplace=True)
        
    # Fonction pour nettoyer et convertir les valeurs de 'superficie' en format float
    def clean_superficie(superficie):
        # Retirer toutes les occurrences de "m2" et extraire les chiffres
        superficie = re.sub(r'\s*m2\s*', '', str(superficie), flags=re.IGNORECASE)
        # Convertir en float
        try:
            return float(superficie)
        except ValueError:
            return None
    # Appliquer la fonction de nettoyage sur la colonne 'superficie' et modification du nom de la colonne
    if 'superficie' in df.columns:
        df['superficie_m2'] = df['superficie'].apply(clean_superficie)
        df.drop(columns=['superficie'], inplace=True)
        
    # Fonction pour nettoyer la colonne 'nb_salle_de_bain'
    def clean_nb_salle_de_bain(nb_salle_de_bain):
        if pd.isna(nb_salle_de_bain):
            return None
        # Utiliser une expression régulière pour extraire le nombre
        match = re.search(r'\d+', nb_salle_de_bain)
        if match:
            return int(match.group())
        return None
    # Convertir toute la colonne 'nb_salle_de_bain' en string avant de faire le traitement
    if 'nb_salle_de_bain' in df.columns:
        df['nb_salle_de_bain'] = df['nb_salle_de_bain'].astype(str)
        df['nb_salle_de_bain'] = df['nb_salle_de_bain'].apply(clean_nb_salle_de_bain)
    # Convertir la colonne 'nb_salle_de_bain' en type Int64
    df['nb_salle_de_bain'] = df['nb_salle_de_bain'].astype('Int64')


    # Fonction pour nettoyer la colonne 'nb_pieces' et la convertir en int
    def clean_nb_pieces(nb_pieces):
        if pd.isna(nb_pieces):
            return None
        # Extraire les chiffres de la chaîne de caractères et les convertir en entier
        match = re.search(r'\d+', str(nb_pieces))
        if match:
            return int(match.group())
        return None

    # Appliquer la fonction pour nettoyer 'nb_pieces'
    df['nb_pieces'] = df['nb_pieces'].apply(clean_nb_pieces)


    def extract_nb_pieces_if_different(row):
        # Vérifier si 'title' est bien une chaîne de caractères
        if isinstance(row['title'], str):
            # Extraire le nombre de pièces du titre
            match = re.search(r'\b(\d+)\s*(pi[eè]ces?|pcs?)\b', row['title'])
            if match:
                nb_pieces_title = int(match.group(1))
                # Vérifier si le nombre de pièces extrait du titre est différent de celui existant
                if row['nb_pieces'] != nb_pieces_title:
                    return nb_pieces_title
            # Cas particulier pour "studio" ou "chambre salon"
            if "studio" in row['title'].lower() or "chambre salon" in row['title'].lower():
                return 1
        return row['nb_pieces']

    # Appliquer la fonction sur chaque ligne du DataFrame
    if 'title' in df.columns and 'nb_pieces' in df.columns:
        df['nb_pieces'] = df.apply(extract_nb_pieces_if_different, axis=1)

    # Convertir 'nb_pieces' en int (si cela est nécessaire)
    df['nb_pieces'] = df['nb_pieces'].astype('Int64')


    def imputer_nb_pieces(df):
        # Définir le motif regex pour trouver le nombre de pièces
        motif = re.compile(r'\b(\d+)\s*(pi[eè]ces?|pcs?)\b')

        def imputer_ligne(row):
            # Vérifier si 'nb_pieces' est manquant
            if pd.isna(row['nb_pieces']):
                # Rechercher le motif dans la description
                match = motif.search(str(row['description']))
                if match:
                    # Extraire et retourner le nombre de pièces
                    row['nb_pieces'] = int(match.group(1))
            return row

        # Appliquer la fonction pour imputer les valeurs manquantes de 'nb_pieces'
        df = df.apply(imputer_ligne, axis=1)
        return df

    # Appliquer la fonction au dataframe
    df = imputer_nb_pieces(df)

    df = df.dropna(subset=['title'])
    df = df.dropna(subset=['nb_pieces'])
    
    df = df[(df['loyer_mensuel'] <= 600000) & (df['loyer_mensuel'] >= 20000)]

    
    print(f'Restants : {len(df)}')


    print("2.6.	Suppression des doublons")
    # Fonction pour gérer les doublons primaires
    def remove_primary_duplicates(df):
        primary_columns = [col for col in df.columns if col not in ['scraping_date', 'date', 'link']]
        duplicates = df[df.duplicated(subset=primary_columns, keep=False)]
        primary_duplicates_count = duplicates.shape[0]
        print(f"Nombre de doublons primaires détectés: {primary_duplicates_count}")
        df = df.drop_duplicates(subset=primary_columns, keep='first')
        return df, duplicates

    # Appliquer la suppression des doublons primaires
    df, primary_duplicates = remove_primary_duplicates(df)


    # Fonction pour gérer les doublons secondaires
    def remove_secondary_duplicates(df):
        df['description_clean'] = df['description'].apply(lambda x: re.sub(r'\W+', ' ', str(x)))
        duplicates = set()
        seen = set()

        for i, row in df.iterrows():
            if i in seen:
                continue
            for j, other_row in df.iterrows():
                if i != j and j not in seen:
                    similarity = fuzz.token_set_ratio(row['description_clean'], other_row['description_clean'])
                    if similarity > 99.5:
                        duplicates.add(j)
                        seen.add(j)
                        break  # Exit the inner loop to ensure only one duplicate is removed

        secondary_duplicates_count = len(duplicates)
        print(f"Nombre de doublons secondaires détectés: {secondary_duplicates_count}")

        duplicates_list = list(duplicates)
        secondary_duplicates = df.loc[duplicates_list]
        df = df.drop(duplicates_list)
        df.drop(columns=['description_clean'], inplace=True)
        return df, secondary_duplicates

    # Appliquer la suppression des doublons secondaires
    df, secondary_duplicates = remove_secondary_duplicates(df)

    print(f'Restants {len(df)}')


    print("2.7.	Création des granularités de la localisation : nouvelles variables de localisation")

    # Dictionnaire pour les grappes et leurs régions associées
    grappe_to_region = {
        'abidjan': ['abidjan'],
        'nord': [r'bafing', r'bagoué', r'folon', r'kabadougou', r'poro', r'tchologo', r'worodougou'],
        'centre': [r'bélier', r'gbêkêe', r'hambol', r'iffou', r'marahoué', r'moronou', r'n’zi', r'yamoussoukro'],
        'est': [r'bounkani', r'gontougo', r'indénié-djuablin', r'mé', r'sud-comoé', r'zanzan'],
        'ouest': [r'agnéby-tiassa', r'béré', r'cavally', r'gbôklé', r'gôh', r'grands-ponts', r'guémon', r'haut-sassandra', r'lôh-djiboua', r'nawa', r'san-pédro', r'tonkpi']
    }


    region_to_ville = {
        # Région d'Abidjan
        'abidjan': ['abidjan'],
        # Autres régions
        'bafing': ['touba'],
        'bagoué': [r'boundiali'],
        'folon': ['minignan'],
        'kabadougou': [r'odienn(é|e)'],
        'poro': ['korhogo'],
        'tchologo': [r'ferkess(é|e)dougou'],
        'worodougou': [r's(é|e)gu(é|e)la'],
        'bélier': [r'yamoussoukro'],
        'gbêkê': [r'bouak(é|e)'],
        'hambol': [r'katiola'],
        'iffou': [r'daoukro'],
        'marahoué': [r'bouafl(é|e)'],
        'moronou': [r'bongouanou'],
        'n’zi': [r'dimbokro'],
        'bounkani': [r'bouna'],
        'gontougo': [r'bondoukou'],
        'indénié-djuablin': [r'abengourou'],
        'mé': [r'adzop(é|e)'],
        'sud-comoé': [r'(bassam|aboisso)'],
        'agnéby-tiassa': [r'agboville'],
        'béré': [r'mankono'],
        'cavally': [r'guiglo'],
        'gbôklé': [r'sassandra'],
        'gôh': [r'gagnoa'],
        'grands-ponts': [r'dabou'],
        'guémon': [r'du(é|e)kou(é|e)'],
        'haut-sassandra': [r'daloa'],
        'lôh-djiboua': [r'divo'],
        'nawa': [r'soubr(é|e)'],
        'san-pédro': [r'san-p(é|e)dro'],
        'tonkpi': [r'man']
    }

    # Dictionnaire pour les villes et leurs communes (seulement pour Abidjan)
    ville_to_commune = {
        'abidjan': ['abobo', 'adjamé', 'attécoubé', 'cocody', 'koumassi', 'marcory', 'plateau', 'port-bouët', 'treichville', 'yopougon', 'anyama', 'bingerville', 'songon']
    }

    # Dictionnaire pour les communes et leurs quartiers associés (seulement pour Abidjan)
    commune_to_quartier = {
        # Abobo
        'abobo': [r'sogephia', r'avocatier', r'abobo doum(é|e)', r'pk18', r'anonkoua kout(é|e)', r'avocatier', r'belleville', r'derri(è|e)re rails', r'n’dotr(é|e)', r'samak(é|e)', r'anador'],
        # Adjamé
        'adjamé': [r'williamsville', r'bracodi', r'ind(é|e)ni(é|e)', r'220 logements', r'adjam(é|e) village', r'libert(é|e)', r'habitat', r'fraternit(é|e)'],
        # Attécoubé
        'attécoubé': [r'banco', r'abobodoum(é|e)', r'locodjro', r'mossikro', r'agban', r'anonkoua', r'att(é|e)coub(é|e) village'],
        # Cocody
        'cocody': [r'cnps', r'rti', r'cocody centre', r'9eme tranche', r'danga', r'angré', r'angre', r'abatta', r'riviera', r'riviéra', r'rivi(é|e)ra 1', r'rivi(é|e)ra 2', r'rivi(é|e)ra 3', r'rivi(é|e)ra 4', r'riviera4',r'palmeraie', r'deux plateaux', r'2 plateaux', r'ii plateaux', r'danga', r'mermoz', r'vallon', r'cité des arts', r'cité rouge', r'ambassade', r'golf', r'faya', 'bonoumin', 'attoban', 'lycée technique', 'riviera triangle', 'abatta', 'école de police', 'ecole de police', 'm\'badon', 'riviera4'],
        # Koumassi
        'koumassi': [r'divo', r'remblais', r'prodomo', r'campement', r'sicogi', r'grand campement', r'zone industrielle', r'addoha'],
        # Marcory
        'marcory': [r'marcory résidentiel', r'orca deco', r'zone 4', r'bietry', r'biétry', r'anoumabo', r'hibiscus', r'r(é|e)sidentiel', r'zone 3', r'zone 4c', r'champroux', r'zone4'],
        # Le Plateau
        'plateau': [r'plateau ind(é|e)ni(é|e)', r'plateau dokui', r'plateau vallon', r'plateau centre', r'plateau nord', r'plateau sud', r'plateau',  r'zone ccia'],
        # Port-Bouët
        'port-bouët': [r'anani', r'gonzagueville', r'vridi', r'adjouffou', r'jean folly', r'port', r'petit bassam', r'vridi canal'],
        # Treichville
        'treichville': [r'arras', r'belleville', r'zone 3', r'avenue 16', r'avenue 8', r'avenue 12', r'avenue 21'],
        # Yopougon
        'yopougon': [r'yopougon carrefour chu', r'ananeraie', r'niangon', r'sid(é|e)ci', r'toits rouges', r'andokoi', r'selmer', r'maroc', r'kout(é|e)', r'wassakara', r'sicogi', r'gesco', r'yopougon cite verte', r'yopougon cité verte', r'yopougon academie', r'yopougon académie']
    }

 
    def process_location_info(df):
        # Fonction pour extraire les informations de localisation (grappe, région, ville)
        def extract_location_info(text):
            tokens = re.findall(r'\b\w+\b', text.lower())
            grappe, region, ville, commune, quartier = None, None, None, None, None

            # 1. Recherche du quartier et déduction de la commune, de la ville, et de la région pour Abidjan
            for commune_key, quartiers in commune_to_quartier.items():
                for quartier_key in quartiers:
                    quartier_tokens = re.findall(r'\b\w+\b', quartier_key.lower())
                    if all(token in tokens for token in quartier_tokens):
                        quartier = quartier_key
                        commune = commune_key
                        ville = 'abidjan'
                        region = 'abidjan'
                        grappe = 'abidjan'
                        return grappe, region, ville, commune, quartier

            # 2. Recherche de la commune et déduction de la ville et de la région pour Abidjan
            for commune_key in ville_to_commune['abidjan']:
                commune_tokens = re.findall(r'\b\w+\b', commune_key.lower())
                if all(token in tokens for token in commune_tokens):
                    commune = commune_key
                    ville = 'abidjan'
                    region = 'abidjan'
                    grappe = 'abidjan'
                    return grappe, region, ville, commune, None

            # 3. Recherche de la ville et déduction de la région
            for region_key, villes in region_to_ville.items():
                for ville_key in villes:
                    ville_tokens = re.findall(r'\b\w+\b', ville_key.lower())
                    if all(token in tokens for token in ville_tokens):
                        ville = ville_key
                        region = region_key
                        for grappe_key, regions in grappe_to_region.items():
                            if region in regions:
                                grappe = grappe_key
                        return grappe, region, ville, None, None

            # Retourner None pour toutes les valeurs si rien n'a été trouvé
            return None, None, None, None, None

        # Appliquer la fonction d'extraction sur chaque ligne du DataFrame
        df['grappe'], df['region'], df['ville'], df['commune'], df['quartier'] = zip(*df.apply(
            lambda row: extract_location_info(f"{row['title']} {row['localisation']} {row['description']}"), axis=1))

        # Fonction pour corriger la commune à partir de la description
        def extract_commune_from_description(description):
            tokens = re.findall(r'\b\w+\b', description.lower())
            for ville, communes in ville_to_commune.items():
                for commune in communes:
                    commune_tokens = re.findall(r'\b\w+\b', commune.lower())
                    if all(token in tokens for token in commune_tokens):
                        return commune
            return None

        # Mise à jour de la commune si elle est vide et que la description peut en fournir une
        df['commune'] = df.apply(lambda row: extract_commune_from_description(row['description']) if pd.isna(row['commune']) else row['commune'], axis=1)

        # Fonction pour corriger la commune à partir du quartier
        def extract_commune_from_quartier(row):
            quartier = row['quartier']
            commune = row['commune']
            if pd.notna(quartier) and pd.isna(commune):
                for commune_key, quartiers in commune_to_quartier.items():
                    quartier_tokens = re.findall(r'\b\w+\b', quartier.lower())
                    for quartier_key in quartiers:
                        quartier_key_tokens = re.findall(r'\b\w+\b', quartier_key.lower())
                        if all(token in quartier_tokens for token in quartier_key_tokens):
                            return commune_key
            return commune

        # Appliquer la fonction pour corriger la commune à partir du quartier
        df['commune'] = df.apply(extract_commune_from_quartier, axis=1)

        # Fonction pour corriger la ville, la région, et la grappe à partir de la localisation si la commune est vide
        def correct_ville_region_grappe_from_localisation(row):
            localisation = row['localisation']
            commune = row['commune']
            ville = row['ville']
            region = row['region']
            grappe = row['grappe']
            if pd.isna(commune) and pd.notna(localisation):
                tokens = re.findall(r'\b\w+\b', localisation.lower())
                for region_key, villes in region_to_ville.items():
                    for ville_key in villes:
                        ville_tokens = re.findall(r'\b\w+\b', ville_key.lower())
                        if all(token in tokens for token in ville_tokens):
                            ville = ville_key
                            region = region_key
                            for grappe_key, regions in grappe_to_region.items():
                                if region in regions:
                                    grappe = grappe_key
                            return ville, region, grappe
            return ville, region, grappe

        # Appliquer la correction ville/région/grappe si commune est vide
        df[['ville', 'region', 'grappe']] = df.apply(lambda row: pd.Series(correct_ville_region_grappe_from_localisation(row)), axis=1)

        # Fonction pour corriger le quartier à partir de la concaténation de localisation et description si quartier est vide
        def correct_quartier_from_localisation_description(row):
            localisation = row['localisation']
            description = row['description']
            quartier = row['quartier']
            commune = row['commune']
            if pd.isna(quartier) and (pd.notna(localisation) or pd.notna(description)) and pd.notna(commune):
                combined_text = f"{localisation} {description}".lower()
                tokens = re.findall(r'\b\w+\b', combined_text)
                if commune in commune_to_quartier:
                    for quartier_key in commune_to_quartier[commune]:
                        quartier_tokens = re.findall(r'\b\w+\b', quartier_key.lower())
                        if all(token in tokens for token in quartier_tokens):
                            return quartier_key
            return quartier

        # Appliquer la correction du quartier si nécessaire
        df['quartier'] = df.apply(lambda row: correct_quartier_from_localisation_description(row), axis=1)

        return df

    # Appliquer la fonction sur le DataFrame
    df = process_location_info(df)


    # Afficher le nombre de lignes restantes
    print(f'Restants : {len(df)}')



    print("2.8.	Correction du type d’immobilier à partir de la variable « description » ")

    if 'type_immobilier' in df.columns:
        df['type_immobilier'] = pd.NA

    # Fonction pour déterminer le type d'immobilier
    def classify_immobilier(row):
        text = str(row['description']) if pd.notna(row['description']) else ''
        text2 = str(row['title']) if pd.notna(row['title']) else ''
        
        nb_pieces = row['nb_pieces']
        
        # Vérifier si nb_pieces n'est pas manquant
        if pd.notna(nb_pieces) and nb_pieces == 1:
            return 'studio'
        
        # Identifier d'autres types d'immobilier à partir de la description (text)
        if re.search(r'duplex', text, re.IGNORECASE):
            return 'duplex'
        if re.search(r'villa', text, re.IGNORECASE):
            return 'villa'
        if re.search(r'\b(appartement|penthouse)\b', text, re.IGNORECASE):
            return 'appartement'
        
        # Si non trouvé dans la description, chercher dans le titre (text2)
        if re.search(r'duplex', text2, re.IGNORECASE):
            return 'duplex'
        if re.search(r'villa', text2, re.IGNORECASE):
            return 'villa'
        if re.search(r'\b(appartement|penthouse)\b', text2, re.IGNORECASE):
            return 'appartement'
        
        # Par défaut, classifier comme 'maison'
        return 'maison'

    # Appliquer la fonction pour créer la colonne 'type_immobilier'
    if 'type d\'immobilier' in df.columns:
        df['type_immobilier'] = df.apply(classify_immobilier, axis=1)

        # Afficher quelques lignes pour vérification avant suppression
        print(df[['type d\'immobilier', 'type_immobilier']].head())
        
        # Supprimer la colonne 'type d\'immobilier' si tout est correct
        df.drop(columns=['type d\'immobilier'], inplace=True)

    df = df.dropna(subset=['type_immobilier'])

    print(f'Restants : {len(df)}')


    print(" 2.9.	Création de variables de classification des variétés (eau, électricité, type de matériaux, toilette interne ou externe, cours commune, quartiers_chics, nb_pieces_classes)")

    # Fonction pour déterminer si les toilettes sont internes
    def has_toilettes_internes(description):
        return 0 if re.search(r"(toilette|wc|douche|salle d'eau) externe", description, re.IGNORECASE) else 1

    # Fonction pour déterminer si la cour est commune
    def has_cours_commune(description):
        return 1 if re.search(r"cours commune", description, re.IGNORECASE) else 0

    # Appliquer les fonctions pour créer les nouvelles variables
    df['Avec_eau'] = 1
    df['Avec_electricite'] = 1
    df['Materiaux_en_dur'] = 1
    df['Toilettes_internes'] = df['description'].apply(has_toilettes_internes)
    df['cours_commune'] = df['description'].apply(has_cours_commune)


    # Définir la liste des quartiers chics
    quartiers_chics = [
        r'angré', 
        r'riviera', 
        r'riviera 1', 
        r'riviera 2', 
        r'riviera 3', 
        r'riviera 4', 
        'palmeraie', 
        'deux plateaux', 
        'danga', 
        'mermoz', 
        r'vallon', 
        'cité des arts', 
        'zone 4', 
        'biétry', 
        'hibiscus', 
        'résidentiel', 
        'plateau',
        'le plateau',
        'ambassade',
        'golf',
        'faya',
        'residentiel'
    ]

    # Fonction pour déterminer si la localisation est dans un quartier chic
    def is_quartier_chic(quartier):
        if pd.isna(quartier): 
            return 0
        quartier = str(quartier)
        for quartier1 in quartiers_chics:
            if re.search(quartier1, quartier, re.IGNORECASE):
                return 1
        return 0

    # Appliquer la fonction pour créer la variable 'quartier_chic'
    df['quartier_chic'] = df['quartier'].apply(is_quartier_chic)

    # Fonction pour classifier le nombre de pièces
    def classify_nb_pieces(nb_pieces):
        if pd.isna(nb_pieces):
            return None
        elif nb_pieces == 1:
            return '1 piece'
        elif 2 <= nb_pieces <= 3:
            return '2 et 3 pieces'
        elif nb_pieces >= 4:
            return '4 pieces et plus'
        else:
            return None

    # Appliquer la fonction pour créer la colonne 'nb_pieces_classe'
    df['nb_pieces_classe'] = df['nb_pieces'].apply(classify_nb_pieces)


    print(f'Restants : {len(df)}')



    # Définir les dictionnaires de mots regroupés
    commodities = {
        'wifi': [r'wifi', r'internet', r'connexion', r'fibre optique'],
        'jardin': [r'jardin(\.locat\d+)?'],
        'meubler': [r'meubl(er|é|e)'],
        'climatisation': [r'climat(isé|isation|iseur)', r'air conditionner', r'split'],
        'garage': [r'garage'],
        'balcon': [r'balcon'],
        'sécuriser': [r'sécur(is|it|is)(er|e|é)'],
        'étage': [r'étage'],
        'parking': [r'parking'],
        'douche': [r'douche'],
        'séjour': [r'séjour', r'sejour'],
        'équiper': [r'équip(er|é|e|ée|ement)'],
        'placard': [r'placard'],
        'salle': [r'salle'],
        'manger': [r'manger'],
        'bureau': [r'bureau'],
        'piscine': [r'piscine'],
        'gardien': [r'gardi(en|en)', r'garde'],
        'électrogène': [r'électrogène'],
        'four': [r'four'],
        'cinéma': [r'cinéma', r'netflix'],
        'ascenseur': [r'ascenseur'],
        'double vitrage': [r'double vitrage'],
        'terrasse': [r'terrasse'],
        'toilette': [r'toilette', r'wc'],
        'buanderie': [r'buanderie'],
        'chauffe': [r'chauff(e|e-eau)'],
        'cuisine': [r'cuisine']
    }

    natural_externalities = {
        'bord_de_mer': [r'plage', r'mer', r'océan'],
        'forêt': [r'for(ê|e)t'],
        'parc': [r'parc'],
        'montagne': [r'montagne'],
        'rivière': [r'rivi(è|e)re'],
        'lac': [r'lac']
    }

    artificial_externalities = {
        'zone_commerciale': [r'centre commercial', r'marché', r'zone commerciale'],
        'pharmacie': [r'pharmacie'],
        'restaurant': [r'restaurant', r'resto'],
        'bar': [r'bar', r'maquis'],
        'ambassade': [r'ambassade'],
        'école': [r'écol(e|es)', r'crèch(e|es)', r'lycé(e|es)', r'universit(é|és)'],
        'hôpital': [r'hôpital', r'clinique', r'dispensaire', r'CHU', r'CHR', r'medecin']
    }

    # Fonction pour extraire les informations de la description
    def extract_description_info(df, grouped_words):
        def extract_info(description):
            if not isinstance(description, str):
                return {key: 0 for key in grouped_words}
            
            doc = nlp(description)
            info = {key: 0 for key in grouped_words}
            
            negation = False
            for token in doc:
                if token.dep_ == 'neg':
                    negation = True
                for key, patterns in grouped_words.items():
                    for pattern in patterns:
                        if re.search(pattern, token.lemma_):
                            info[key] = 0 if negation else 1
                            negation = False
            
            return info
        
        description_info = df['description'].apply(extract_info)
        
        for key in grouped_words.keys():
            df[key] = description_info.apply(lambda x: x[key])
        
        return df

    # Appliquer les fonctions pour extraire les informations des commodités et externalités
    df = extract_description_info(df, commodities)
    df = extract_description_info(df, natural_externalities)
    df = extract_description_info(df, artificial_externalities)

    # Créer les variables 'commodités', 'externalités_naturelles' et 'externalités_artificielles'
    df['commodités'] = df[list(commodities.keys())].max(axis=1)
    df['externalités_naturelles'] = df[list(natural_externalities.keys())].max(axis=1)
    df['externalités_artificielles'] = df[list(artificial_externalities.keys())].max(axis=1)


    # Exporter le dataframe final avec les modifications
    import platform
    if platform.system() == 'Windows':
        output_path_final = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Traitements\\Datasets_pretraites\\pretraitees\\{mois}_pretraitee.xlsx'
    else:
        output_path_final = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Traitements/Datasets_pretraites/{mois}_pretraitee.xlsx'

    df.to_excel(output_path_final, index=False)

    print(f"Fichier final avec les modifications sauvegardé sous {output_path_final}")

Les traitements sont en cours pour le mois de Mai, cela pourrait prendre quelques minutes...
Restants : 3998
2.2.	Convertir toutes les données en minuscules pour faciliter le traitement des données textuelles
Restants : 3966
2.3. Création des variables « site » et « date »
2.4.	Suppression des lignes non pertinentes pour l’étude (vente, entrepôt, terrains, etc.)
Restants : 797
2.5.	Numérisation des variables quantitatives (nombre de pièces, nombre de salle de bain, loyer_mensuel et superficie en m2)


C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:116: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['concat_description_title'].str.contains(daily_and_no_location, case=False, na=False)]
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:125: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['title'].str.contains(multiple_properties_pattern, case=False, na=False)]


Restants : 484
2.6.	Suppression des doublons
Nombre de doublons primaires détectés: 206
Nombre de doublons secondaires détectés: 18
Restants 292
2.7.	Création des granularités de la localisation : nouvelles variables de localisation
Restants : 292
2.8.	Correction du type d’immobilier à partir de la variable « description » 
   type d'immobilier type_immobilier
0       appartements          duplex
2       appartements          maison
3       appartements          maison
8       appartements     appartement
11      appartements          maison
Restants : 292
 2.9.	Création de variables de classification des variétés (eau, électricité, type de matériaux, toilette interne ou externe, cours commune, quartiers_chics, nb_pieces_classes)
Restants : 292
Fichier final avec les modifications sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques\Traitements\Datasets_pretraites\pretraitees\Mai_pretraitee.xlsx
Les traitements sont en cours pour le mois de Juin, cela pourrait prendre quelques m

C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:116: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['concat_description_title'].str.contains(daily_and_no_location, case=False, na=False)]
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:125: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['title'].str.contains(multiple_properties_pattern, case=False, na=False)]


Restants : 2408
2.5.	Numérisation des variables quantitatives (nombre de pièces, nombre de salle de bain, loyer_mensuel et superficie en m2)
Restants : 1679
2.6.	Suppression des doublons
Nombre de doublons primaires détectés: 1159
Nombre de doublons secondaires détectés: 59
Restants 656
2.7.	Création des granularités de la localisation : nouvelles variables de localisation
Restants : 656
2.8.	Correction du type d’immobilier à partir de la variable « description » 
   type d'immobilier type_immobilier
0       appartements          maison
1       appartements          maison
6       appartements     appartement
9       appartements          maison
11      appartements          maison
Restants : 656
 2.9.	Création de variables de classification des variétés (eau, électricité, type de matériaux, toilette interne ou externe, cours commune, quartiers_chics, nb_pieces_classes)
Restants : 656
Fichier final avec les modifications sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques\Trait

C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:116: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['concat_description_title'].str.contains(daily_and_no_location, case=False, na=False)]
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:125: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['title'].str.contains(multiple_properties_pattern, case=False, na=False)]


Restants : 5040
2.5.	Numérisation des variables quantitatives (nombre de pièces, nombre de salle de bain, loyer_mensuel et superficie en m2)
Restants : 3251
2.6.	Suppression des doublons
Nombre de doublons primaires détectés: 1200
Nombre de doublons secondaires détectés: 209
Restants 2298
2.7.	Création des granularités de la localisation : nouvelles variables de localisation
Restants : 2298
2.8.	Correction du type d’immobilier à partir de la variable « description » 
   type d'immobilier type_immobilier
56       appartement          maison
66       appartement     appartement
72       appartement     appartement
74       appartement     appartement
76       appartement          duplex
Restants : 2298
 2.9.	Création de variables de classification des variétés (eau, électricité, type de matériaux, toilette interne ou externe, cours commune, quartiers_chics, nb_pieces_classes)
Restants : 2298
Fichier final avec les modifications sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques\

C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:116: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['concat_description_title'].str.contains(daily_and_no_location, case=False, na=False)]
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_102948\3559000626.py:125: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['title'].str.contains(multiple_properties_pattern, case=False, na=False)]


Restants : 4541
2.5.	Numérisation des variables quantitatives (nombre de pièces, nombre de salle de bain, loyer_mensuel et superficie en m2)
Restants : 3022
2.6.	Suppression des doublons
Nombre de doublons primaires détectés: 2814
Nombre de doublons secondaires détectés: 66
Restants 812
2.7.	Création des granularités de la localisation : nouvelles variables de localisation
Restants : 812
2.8.	Correction du type d’immobilier à partir de la variable « description » 
    type d'immobilier type_immobilier
25            maisons           villa
43            maisons          studio
44            maisons           villa
66            maisons          maison
125           maisons     appartement
Restants : 812
 2.9.	Création de variables de classification des variétés (eau, électricité, type de matériaux, toilette interne ou externe, cours commune, quartiers_chics, nb_pieces_classes)
Restants : 812
Fichier final avec les modifications sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques